In [1]:
from dotenv import load_dotenv
from groq import Groq

import os

load_dotenv('../.env')


client = Groq(
  api_key=os.getenv("GROQ_API_KEY")
)

MODEL = "llama-3.3-70b-versatile"

# test Groq connection
chat_completion = client.chat.completions.create(
  messages=[
  {
    "role": "system",
    "content": "You are a helpful assistant."
  },
  {
    "role": "user",
    "content": "Explain the importance of fast language models",
  }
  ],
  model=MODEL,
  max_completion_tokens=250
)

print(chat_completion.choices[0].message.content)

Fast language models are a crucial component of modern natural language processing (NLP) systems. The importance of fast language models can be understood from several perspectives:

1. **Improved User Experience**: Fast language models enable real-time interaction with users, allowing them to receive immediate responses to their queries. This leads to a more engaging and efficient user experience, particularly in applications such as chatbots, virtual assistants, and language translation software.
2. **Increased Productivity**: By processing and responding to language inputs quickly, fast language models can significantly boost productivity in various industries, such as customer service, content creation, and data analysis. This enables businesses to automate tasks, reduce response times, and enhance overall efficiency.
3. **Enhanced Accuracy**: Fast language models can be trained on large datasets and fine-tuned to achieve high accuracy in tasks such as language translation, sentime

In [3]:
import random
import json

known_weather_data = {
  'berlin': 20.0
}

def get_weather(city: str) -> float:
  city = city.strip().lower()

  if city in known_weather_data:
      return known_weather_data[city]

  return round(random.uniform(-5, 35), 1)

get_weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Retrieves the temperature for a specified city.",
        "parameters": {
            "type": "object",
            "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city, e.g. New York"
            }
            },
            "required": ["city"],
            "additionalProperties": False
        }
    }
}

# imports calculate function from step 1
def run_conversation(user_prompt):
    # Initialize the conversation with system and user messages
    messages=[
        {
            "role": "system",
            "content": "You are a weather assistant. Use the get weather function to get the weather for a given city and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    # Define the available tools (i.e. functions) for our model to use
    tools = [get_weather_tool]
    # Make the initial API call to Groq
    response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        stream=False,
        tools=tools, # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_completion_tokens=4096 # Maximum number of tokens to allow in our response
    )
    print("response", response)
    # Extract the response and any tool call responses
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        # Define the available tools that can be called by the LLM
        available_functions = {
            "get_weather": get_weather,
        }
        # Add the LLM's response to the conversation
        messages.append(response_message)

        # Process each tool call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            # Call the tool and get the response
            function_response = function_to_call(
                city=function_args.get("city")
            )
            function_response = str(function_response)
            # Add the tool response to the conversation
            messages.append(
                {
                    "tool_call_id": tool_call.id, 
                    "role": "tool", # Indicates this message is from tool use
                    "name": function_name,
                    "content": function_response,
                }
            )
        # Make a second API call with the updated conversation
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        print("second_response", second_response)
        # Return the final response
        return second_response.choices[0].message.content
# Example usage
user_prompt = "What is the weather in berlin?"
print(run_conversation(user_prompt))

response ChatCompletion(id='chatcmpl-01de503b-062b-483b-b51d-42da80c732bc', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=[ChatCompletionMessageToolCall(id='zjkvdfvam', function=Function(arguments='{"city":"Berlin"}', name='get_weather'), type='function')]))], created=1752640090, model='llama-3.3-70b-versatile', object='chat.completion', system_fingerprint='fp_9a8b91ba77', usage=CompletionUsage(completion_tokens=14, prompt_tokens=268, total_tokens=282, completion_time=0.045713336, prompt_time=0.014753571, queue_time=0.273749533, total_time=0.060466907), usage_breakdown=None, x_groq={'id': 'req_01k08qtwg9e93b4fqdb471zjqb'}, service_tier='on_demand')
second_response ChatCompletion(id='chatcmpl-de32e6a4-8566-4cd4-b9f8-08c9e9255776', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The cur

In [16]:
import importlib

import chat_assistant
importlib.reload(chat_assistant)


tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

developer_prompt = """
You are a weather assistant. Use the get weather function to get the weather for a given city and provide the results.
Use the set weather function to set the weather for a given city.
If not city name is provided, return 'No city name provided'.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client,
    model=MODEL
)

In [7]:
chat.run()

Chat ended.


In [8]:
def set_weather(city: str, temp: float) -> None:
  city = city.strip().lower()
  known_weather_data[city] = temp
  return 'OK'

set_weather_tool = {
  "type": "function",
  "function": {
    "name": "set_weather",
    "description": "Sets the temperature for a specified city.",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string",
          "description": "The name of the city, e.g. New York"
        },
        "temp": {
          "type": "number",
          "description": "The temperature to associate with the city, e.g. 20"
        }
      },
      "required": ["city", "temp"],
      "additionalProperties": False
    }
  }
}

tools.add_tool(set_weather, set_weather_tool)

tools.get_tools()

[{'type': 'function',
  'function': {'name': 'get_weather',
   'description': 'Retrieves the temperature for a specified city.',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string',
      'description': 'The name of the city, e.g. New York'}},
    'required': ['city'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'set_weather',
   'description': 'Sets the temperature for a specified city.',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string',
      'description': 'The name of the city, e.g. New York'},
     'temp': {'type': 'number',
      'description': 'The temperature to associate with the city, e.g. 20'}},
    'required': ['city', 'temp'],
    'additionalProperties': False}}}]

In [9]:
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client,
    model=MODEL
)
chat.run()

Chat ended.


In [10]:
known_weather_data

{'berlin': 20.0, 'milan': 35}

## MCP

In [17]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully


In [18]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '20.0'}],
 'structuredContent': {'result': 20.0},
 'isError': False}

In [28]:
import importlib

# reload the chat_assistant and mcp_client modules to ensure they are up-to-date
importlib.reload(chat_assistant)
importlib.reload(mcp_client)

our_mcp_client = mcp_client.MCPClient(["python", "server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)
print("mcp_tools", mcp_tools.get_tools())


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client,
    model=MODEL
)

chat.run()

Started server with command: python server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['get_weather', 'set_weather', 'get_known_weather_data']
mcp_tools [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Retrieves the temperature for a specified city.', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'City'}}, 'required': ['city'], 'additionalProperties': False}}}, {'type': 'function', 'function': {'name': 'set_weather', 'description': 'Sets the temperature for a specified city.', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'strin

Calling tool 'set_weather' with arguments: {'city': 'Boston', 'temp': 11}


Calling tool 'get_known_weather_data' with arguments: None


Calling tool 'get_known_weather_data' with arguments: None


Chat ended.
